In [ ]:
from collections import defaultdict
import numpy as np
from pychunkedgraph.graph import ChunkedGraph
from pychunkedgraph.graph.utils.inspect import dfs_print_node

from pychunkedgraph.graph.edits import add_edges, remove_edges
from pychunkedgraph.graph.utils.context_managers import TimeIt
from pychunkedgraph.graph.operation import MergeOperation

cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

In [ ]:
data = [
    ["94595040390827505",909404.9375,930447.875,784063.25],
    ["94524671646636264",909251.0625,930452.1875,784063.25]
]
data = {
    "sources": [data[0]],
    "sinks": [data[1]]
}
data

In [ ]:
# data = {
#     "sources":[["94525358841391819",909299.25,950458.1875,783103.25]],
#     "sinks":[["94595727585563012",909363.3125,950430.3125,783103.25]]
# }

# data = {
#     "sources":[["94525358841391819",909299.25,950458.1875,783103.25]],
#     "sinks":[["94595727585563012",909363.3125,950430.3125,783103.25]]
# }

In [ ]:
data_dict = {}
for k in ["sources", "sinks"]:
    data_dict[k] = defaultdict(list)
    for node in data[k]:
        node_id = node[0]
        x, y, z = node[1:]
        coordinate = np.array([x, y, z]) / cg.meta.resolution
        atomic_id = cg.get_atomic_id_from_coord(
            coordinate[0],
            coordinate[1],
            coordinate[2],
            parent_id=np.uint64(node_id),
        )

        if atomic_id is None:
            raise ValueError("aha")
        data_dict[k]["id"].append(atomic_id)
        data_dict[k]["coord"].append(coordinate)

edges = [
    [data_dict["sources"]["id"][0], data_dict["sinks"]["id"][0]],
]
roots_old = cg.get_roots(edges[0])
parents_old = cg.get_parents(edges[0])
print(cg.get_cross_chunk_edges_layer(edges))
print(edges[0])
print(parents_old)
print(roots_old)
dfs_print_node(cg, roots_old[0], limit=2, sep="    ", count_only=True)
dfs_print_node(cg, roots_old[1], limit=2, sep="    ", count_only=True)

In [ ]:
with TimeIt("add_edges"):
    result = cg.add_edges(
        user_id="hi",
        atomic_edges=edges,
        source_coords=data_dict["sources"]["coord"],
        sink_coords=data_dict["sinks"]["coord"],
    )

In [ ]:
print(result)
roots = result.new_root_ids
dfs_print_node(cg, roots[0], limit=2, sep="    ", count_only=True)